In [107]:
import pandas as pd
import numpy as np
import os
import re
import spacy
import emoji
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BIGNETWORK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\BIGNETWORK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:
path = ('C:/Users/BIGNETWORK/Desktop/PFE/dataset/resaux scraper')
files = os.listdir(path)
files

['alg_telcom_2_2k.xlsx',
 'facebook (1).xlsx',
 'facebook (2).xlsx',
 'instagram (1).xlsx',
 'instagram (2).xlsx',
 'instagram (3).xlsx',
 'instagram (4).xlsx',
 'instagram (5).xlsx',
 'instagram (6).xlsx',
 'instagram (7).xlsx',
 'instagram (8).xlsx',
 'instagram.xlsx']

In [109]:
files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls


['alg_telcom_2_2k.xlsx',
 'facebook (1).xlsx',
 'facebook (2).xlsx',
 'instagram (1).xlsx',
 'instagram (2).xlsx',
 'instagram (3).xlsx',
 'instagram (4).xlsx',
 'instagram (5).xlsx',
 'instagram (6).xlsx',
 'instagram (7).xlsx',
 'instagram (8).xlsx',
 'instagram.xlsx']

In [ ]:
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(f'C:/Users/BIGNETWORK/Desktop/PFE/dataset/resaux scraper/{f}')
    data.drop(data.index[0:0],inplace=True)
    df = df.append(data)

In [111]:
print(df)

                                              comments  class
0    باينة كتبتوها نهار السبت حتى اليوم باش تبارتاج...    NaN
1         لوكان ستنيتو حتى نهار الخميس خير و درتو بيان    NaN
2                       الزبون لا دخل له في البكالوريا    NaN
3    شركة.... تعجز عن وضع أجهزة تشويش بمراكز الإمتح...    NaN
4    دوماج عندي لافامي في الكونت هذا كون عبرتلكم عن...    NaN
..                                                 ...    ...
785                                                  🔥    NaN
786                                               ❤️❤️    NaN
787                                               ❤️❤️    NaN
788                         NChllh  b rebi ping yhbt 😂    NaN
789                                              ❤️❤️😍    NaN

[6834 rows x 2 columns]


In [112]:
df.to_excel("resaux_comments.xlsx")

In [2]:
df0=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/resaux_comments.xlsx")
df1=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/hateonly_mehdi.xlsx")
df2=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/fb_comments.xlsx")
df3=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/hainer_ranim.xlsx")
df4=pd.read_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/datasets/neutre_mehdi.xlsx")

In [3]:
df = pd.concat([df0, df1, df2, df3,df4], axis=0, ignore_index=True)

In [64]:
df

,comments,class,comments_clean
0,Tagdo3o cnx men lhad hta lkhmis wzid jam3a mkn...,NaN,Tagdo3o cnx men lhad hta lkhmis wzid jam3a mkn...
1,و كونيكسيون ديالها رهج و زيرو كارتة و هدا وين ...,NaN,كونيكسيون ديالها رهج زيرو كارتة هدا عرفت 60 gb...
2,Sera9in,NaN,Sera9in
3,هذي الكفار او ماداروهاش,NaN,الكفار ماداروهاش
4,تافهين وخدماتكم دون المستوى وماشكيتش تتحسنو,NaN,تافهين وخدماتكم المستوى وماشكيتش تحسنو
...,...,...,...
10076,تحيا جيزي راهي علامة الشبكة 100/100 متسرقش نكم...,NaN,تحيا جيزي راهي علامة الشبكة 10 10 متسرقش نكمل ...
10077,انا مكان مادخلني فالباك راني نخلص فيها وهذا حقي,NaN,مادخلني فالباك راني نخلص وهذا حقي
10078,iOS !!!,NaN,iOS
10079,الله يالكونكسين تاعكم رجعت تعيف تعيف تعييييف,1.0,اله يالكونكسين تاعكم رجعت تعيف تعيف تعيف


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

In [6]:
df.to_excel("C:/Users/BIGNETWORK/Desktop/PFE/dataset/dataset.xlsx")

In [7]:
df=df.dropna(subset = ['comments'])

In [8]:
df['comments'].isna().sum()

0

*** 
---
___

## <span style='font-family:Georgia'> Preprocessing of Text.

### <span style='font-family:serif'> Function to Remove Emojis

In [13]:
def strip_emoji(text):
    return emoji.replace_emoji(text,replace="")

### <span style='font-family:serif'> removing stop words

In [73]:
# Load stopwords for each language
with open('C:/Users/BIGNETWORK/Desktop/PFE/dataset/project/git-one/Algerian-Arabic-stop-words.txt', 'r', encoding='utf-8') as f:
    stop_words_ar_dz = set([line.strip() for line in f])
stop_words_en = set(stopwords.words('english'))
stop_words_fr = set(stopwords.words('french'))
from nltk.tokenize import RegexpTokenizer
# Create a custom tokenizer for Arabic words
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
# Define a function to remove stop words from a text
def remove_stopwords(text):
    words = tokenizer.tokenize(text)
    words_filtered = []
    for word in words:
        if word not in stop_words_ar_dz and word not in stop_words_en and word not in stop_words_fr:
            words_filtered.append(word)
    return ' '.join(words_filtered)
tet = "Wlh loukan temdou alef jiga batel tqa3dou halazouuun."
tet = remove_stopwords(tet)
print(tet)

loukan temdou alef jiga batel tqa3dou halazouuun .


### <span style='font-family:serif'> Fucntion to remove special characters, URLs, duplicated letters, punctuations

In [55]:
# Define a function to perform the text cleaning operations
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation marks 
    text = re.sub(r'[@#&$]\w+', '', text)  # Remove special characters
    #text = re.sub(r'\d+', '', text)  # Remove numbers 
    text = re.sub(r'(\w)\1+', r'\1', text) # remove duplicated letters
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    return text

'''# Apply the clean_text function to the desired column(s) in the DataFrame
if df['comments'].dtype == 'object':
    df['comments_clean'] = df['comments'].apply(clean_text)'''


"# Apply the clean_text function to the desired column(s) in the DataFrame\nif df['comments'].dtype == 'object':\n    df['comments_clean'] = df['comments'].apply(clean_text)"

### <span style='font-family:serif'> Function to remove mutiple sequence spaces

In [43]:
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)

### <span style='font-family:serif'> Function to Preprocess the text by applying all above functions

In [74]:
def preprocess(text):
    text = strip_emoji(text)
    text = remove_stopwords(text)
    text = clean_text(text)
    text = remove_mult_spaces(text)
    return text


In [75]:
if df['comments'].dtype == 'object':
    df['comments_clean'] = df['comments'].apply(preprocess)

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_13008\2574735620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments_clean'] = df['comments'].apply(preprocess)


In [76]:
df.sample(25)

,comments,class,comments_clean
4042,سكت دهراً .. ونطق كفراً,NaN,سكت دهرا ونطق كفرا
796,سراق يفليكسي لواحد يجي ياكتيفي يلقاها راحت درا...,NaN,سراق يفليكسي لواحد يجي ياكتيفي يلقاها راحت درا...
9875,",ن. م بإلحاح و بكل روح وطنية ن.م ن. م ن. م",NaN,ن م بإلحاح بكل روح وطنية م م م
6274,خلي راني صايم مور فطور نشا الله نتهلا,NaN,خلي راني صايم مور فطور نشا اله نتهلا
4343,هذي طلقتوها يامات لولة تع باك هذا وين فاتت pub...,NaN,طلقتوها يامات لولة باك فات publication تعكم شر...
5161,نخلة هذيك تاع شطيح ورطيح هههههههه خاطي قاع بوب...,NaN,نخلة هذيك شطيح ورطيح ه خاطي قاع بوبجي
4017,Nik ymah l9hba li b9a ychri djezzy ynik mo ykh...,-1.0,Nik ymah l9hba li b9a ychri djezy ynik mo ykhl...
2041,Ani bgratwi yzbi flixsit dawhomli djezzy ta3 zbi,-1.0,Ani bgratwi yzbi flixsit dawhomli djezy ta3 zbi
4169,Dirolna free fire 😭,NaN,Dirolna fre fire
2556,صحا ليندينlinkdedln ثاني يتريشو بيه,NaN,صحا ليندينlinkdedln ثاني يتريشو


In [77]:
df['comments_clean'].isna().sum()

0

In [78]:
df['comments_clean'].replace('', np.nan, inplace=True)
missing_values = df[df['comments_clean'].isnull()]
#df=df.dropna(subset = ['comments_clean'])
len(missing_values)

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_13008\3493959318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments_clean'].replace('', np.nan, inplace=True)


380

In [79]:
df=df.dropna(subset = ['comments_clean'])

In [101]:
from langdetect import detect
from difflib import SequenceMatcher

In [102]:
def lang_det(texte):
    langage = detect(texte)
    if langage not in ['en','fr']:
        langage = 'dz'
    
    return langage

In [111]:
text ='3andkom cnx ma9wda bzfffffff ou rizo bal m nhki'
print(lang_det(text))

en


In [114]:
# Load the language detection pipeline
lang_detect = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Text to classify
text = "Pourquoi vous limitez le débit ?."

# Run the language detection pipeline
result = lang_detect(text)

# Print the detected language
print(result[0]['label'])

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


it


In [121]:
# Text to classify
text = "ڨاع ماشي conection surtout les environ ta3 sedikia"

# Run the language detection pipeline
result = lang_detect(text)

# Print the detected language
print(result[0]['label'])

ar


In [84]:
def similar(a, b):
# Utilisation de la méthode SequenceMatcher pour calculer la similarité entre deux chaînes de caractères
    return SequenceMatcher(None, a, b).ratio()
def detecter_langage(texte):
    # Détection de la langue avec langdetect
    langage = detect(texte)
    
   #  Si le texte est similaire à l'anglais, au français ou à l'arabe, il est classé comme dialecte algérien
    if langage in ['en', 'fr', 'ar']:
        similarites = {
            'en': similar(texte, 'english'),
            'fr': similar(texte, 'français'),
            'ar': similar(texte, 'العربية')
        }
        # Le texte est classé comme dialecte algérien s'il est plus similaire à une de ces langues qu'à l'autre
        if max(similarites.values()) == similarites[langage] or langage =='so':
            langage = 'dz'
    
    return langage

In [105]:
txt = "Kanet 3endi beseh réseau fiha mekhsous bezaf f la willaya de Tizi ouzou apart le centre ville makanch réseau"
print(detecter_langage(txt))
  #  df['len'] = df['comments_nett'].apply(detecter_langage)

fr


In [122]:
for index, row in df.iterrows():
    text = str(row['comments_clean'])
    if text:
        try:
            result = lang_detect(text)
            df.at[index, 'language'] = result[0]['label']
        except:
            pass

C:\Users\BIGNETWORK\AppData\Local\Temp\ipykernel_13008\2026574333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'language'] = result[0]['label']
